<a href="https://colab.research.google.com/github/greyhound101/Brain_tumour/blob/master/trying_job.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive
Mounted at /content/gdrive


In [ ]:
import zipfile
import h5py
from tensorflow.keras.optimizers import *
import cv2
from tensorflow.keras.utils import to_categorical
import glob, os
from matplotlib import pyplot as plt
import h5py
from sklearn.metrics import accuracy_score
import numpy as np
from tqdm import tqdm
import time
import gc
from tensorflow.keras.applications import *
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
import tensorflow as tf
import random, os, sys
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow import keras 
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *

import keras
import pandas as pd
from keras.applications.nasnet import NASNetMobile, preprocess_input
import imgaug as ia
from imgaug import augmenters as iaa

In [ ]:
import zipfile
with zipfile.ZipFile('/content/gdrive/MyDrive/dataset cv ai 2021.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [ ]:
import glob
len(glob.glob('/content/landscapes/*')),len(glob.glob('/content/animals/*')),len(glob.glob('/content/buildings/*'))

(100, 100, 100)

In [37]:
landscapes=glob.glob('/content/landscapes/*')
animals=glob.glob('/content/animals/*')
buildings=glob.glob('/content/buildings/*')
tst_paths=landscapes[:10]+animals[:10]+buildings[:10]
tst_labels=[0]*10+[1]*10+[2]*10
trn_paths=landscapes[10:]+animals[10:]+buildings[10:]
trn_labels=[0]*90+[1]*90+[2]*90

In [38]:
import pandas as pd
df_trn=pd.DataFrame({'paths':trn_paths,'labels':trn_labels})
df_tst=pd.DataFrame({'paths':tst_paths,'labels':tst_labels})
df_trn.shape,df_tst.shape

((270, 2), (30, 2))

In [39]:


import keras
import pandas as pd
from keras.applications.nasnet import NASNetMobile, preprocess_input
import imgaug as ia
from imgaug import augmenters as iaa
import tensorflow as tf
class DataGenerator(tf.keras.utils.Sequence):
  def __init__(self, images, labels,transform, batch_size=64):
    self.labels = labels              # array of labels
    self.images = images        # array of image paths
    self.batch_size   = batch_size          # batch size
    self.on_epoch_end()
    self.transform=transform
  def __len__(self):
    return int(np.floor(self.labels.shape[0] / self.batch_size))

  def on_epoch_end(self):
    self.indexes = np.arange(self.labels.shape[0])

  def __getitem__(self, index):
		# selects indices of data for next batch
    indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
    # select data and load images
    labels = self.labels[indexes]
    img = self.images[indexes]
    images=[]
    for i in img:
      if self.transform!=None:
        images.append(self.transform(image=cv2.imread(i))['image'])
      else:
        images.append(cv2.imread(i))

    labels1 = np.zeros((labels.size, 3))
    labels1[np.arange(labels.size),labels] = 1
    return np.stack(images), labels1
# trn=DataGenerator(df['paths'].values,df['labels'].values,aug)

In [40]:
import albumentations 
aug_trn=albumentations.Compose(
        [   
            albumentations.Resize(224,224,always_apply=True),
            albumentations.HorizontalFlip(p=0.5),
            albumentations.VerticalFlip(p=0.5),
            albumentations.Rotate(limit=120, p=0.8),
            albumentations.RandomBrightness(limit=(0.09, 0.6), p=0.5),
            albumentations.Normalize( mean=[0.485, 0.456, 0.406] , std=[0.229, 0.224, 0.225])
        ]
    )
aug_tst=albumentations.Compose(
        [   
            albumentations.Resize(224,224,always_apply=True),
            albumentations.Normalize( mean=[0.485, 0.456, 0.406] , std=[0.229, 0.224, 0.225])
        ]
    )

In [41]:
from tensorflow.keras.layers import *
import tensorflow as tf
import random, os, sys
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.initializers import *
import tensorflow as tf

In [42]:
from tensorflow import keras
from tensorflow.keras import backend as K
class LayerNormalization(Layer):
    def __init__(self, eps=1e-6, **kwargs):
        self.eps = eps
        super(LayerNormalization, self).__init__(**kwargs)
    def build(self, input_shape):
        self.gamma = self.add_weight(name='gamma', shape=input_shape[-1:],
                                     initializer=Ones(), trainable=True)
        self.beta = self.add_weight(name='beta', shape=input_shape[-1:],
                                    initializer=Zeros(), trainable=True)
        super(LayerNormalization, self).build(input_shape)
    def call(self, x):
        mean = K.mean(x, axis=-1, keepdims=True)
        std = K.std(x, axis=-1, keepdims=True)
        return self.gamma * (x - mean) / (std + self.eps) + self.beta
    def compute_output_shape(self, input_shape):
        return input_shape
class abc(keras.layers.Layer):
    def __init__(self,
                 hidden_dim,
                 **kwargs):
        self.hidden_dim = hidden_dim
        
        self.conv1=Conv1D(self.hidden_dim,1)
        self.conv2=Conv1D(self.hidden_dim,1)
        self.conv3=Conv1D(self.hidden_dim,1)
        
        self.Wq = self.Wk = self.Wv = self.Wo = None
        self.bq = self.bk = self.bv = self.bo = None

        self.intensity = self.attention = None
        super(abc, self).__init__(**kwargs)

    def call(self, inputs, mask=None):
        
        q, k, v = inputs
        
        q=self.conv1(q)
        k=self.conv2(k)
        v=self.conv3(v)
        
        
        def scaled_dot_product_attention(inputs):
          query, key, value = inputs
          feature_dim = K.shape(query)[-1]
          e = K.batch_dot(query, key, axes=2) 
          intensity = e
          e = K.exp(e - K.max(e, axis=-1, keepdims=True))
          attention = e / K.sum(e, axis=-1, keepdims=True)
          v = K.batch_dot(attention, value)
          return v
       
       
        y = scaled_dot_product_attention(inputs=[q,k,v])
        
        
        return y
from tensorflow import keras 
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import *
from tensorflow.keras.applications import *
def load_model():   
  
  K.clear_session() 
  mod=densenet.DenseNet121(include_top=True, weights='imagenet')
  d = mod.get_layer('conv5_block16_concat').output

  inp = mod.get_layer('conv3_block12_concat').output
  a = Reshape((28*28,512))(inp)
  d_a = keras.layers.UpSampling2D(interpolation='bilinear',size=(4,4))(d)
  d_a = Reshape((28*28,1024))(d_a)
  a = abc(hidden_dim=512)([a,d_a,a])
  a = LayerNormalization()(a)
  a = Reshape((28,28,512,))(a)
  a = keras.layers.GlobalAveragePooling2D()(a)

  inp = mod.get_layer('conv4_block24_concat').output
  b = Reshape((14*14,1024))(inp)
  d_b = keras.layers.UpSampling2D(interpolation='bilinear',size=(2,2))(d)
  d_b = Reshape((14*14,1024))(d_b)
  b = abc(hidden_dim=1024)([b,d_b,b])
  b = LayerNormalization()(b)
  b = Reshape((14,14,1024,))(b)
  b = keras.layers.GlobalAveragePooling2D()(b)

  d = keras.layers.GlobalAveragePooling2D()(d)
  

  b = Dense(3, activation="softmax")(b) 
  b = Reshape((-1,3))(b) 
  a = Dense(3, activation="softmax")(a) 
  a = Reshape((-1,3))(a) 
  d = Dense(3, activation="softmax")(d) 
  d = Reshape((-1,3))(d) 
  
  conc=Concatenate(axis=1)([a,b,d])
  conc=keras.layers.GlobalAveragePooling1D()(conc)
  mod=Model(inputs=mod.input,outputs=conc)
  return mod

In [43]:
mod=load_model()
mod.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
______________________________________________________________________________________________

In [44]:
from sklearn.model_selection import StratifiedKFold
from collections import Counter
skf = StratifiedKFold(n_splits=5,random_state=42)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [ ]:

for index,(x,y) in enumerate(skf.split(df_trn['paths'], df_trn['labels'])):
  print(Counter(df_trn.loc[y]['labels']))
  print(Counter(df_trn.loc[x]['labels']))
  index=1
  epoch=100
  pre_acc=0
  best=0
  fold='fold_'+str(index)
  trn,tst=df_trn.loc[x].reset_index(drop=True),df_trn.loc[y].reset_index(drop=True)
  tst_y=tst['paths'].values
  
  model=load_model()
  model.compile(optimizer=Adam(1e-5,decay=1e-5), 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])
  train_data=DataGenerator(trn['paths'].values,trn['labels'].values,aug_trn,batch_size=4)
  tst=DataGenerator(tst['paths'].values,tst['labels'].values,aug_tst,batch_size=4)
  ln=len(train_data)
  #fitting the model
  #timing
  es=tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=10,
    mode="min",
    restore_best_weights=True,
  )

  start=time.time()
  hist=model.fit_generator(train_data,validation_data=tst,epochs=epoch,steps_per_epoch=ln//4,callbacks=[es])
  model.save_weights('/content/gdrive/MyDrive/weights_'+str(index)+'.hdf5')

Counter({0: 18, 1: 18, 2: 18})
Counter({0: 72, 1: 72, 2: 72})


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
13/13 [==============================] - 15s 297ms/step - loss: 1.2487 - accuracy: 0.1346 - val_loss: 1.0943 - val_accuracy: 0.3462
Epoch 2/100
13/13 [==============================] - 2s 145ms/step - loss: 1.1548 - accuracy: 0.3077 - val_loss: 1.0707 - val_accuracy: 0.5000
Epoch 3/100
13/13 [==============================] - 2s 146ms/step - loss: 1.0972 - accuracy: 0.3077 - val_loss: 1.0752 - val_accuracy: 0.3462
Epoch 4/100
13/13 [==============================] - 2s 144ms/step - loss: 1.2249 - accuracy: 0.2500 - val_loss: 1.0547 - val_accuracy: 0.4038
Epoch 5/100
13/13 [==============================] - 2s 144ms/step - loss: 1.1364 - accuracy: 0.2115 - val_loss: 1.0453 - val_accuracy: 0.5192
Epoch 6/100
13/13 [==============================] - 2s 147ms/step - loss: 1.0797 - accuracy: 0.4423 - val_loss: 1.0580 - val_accuracy: 0.4615
Epoch 7/100
 4/13 [========>.....................] - ETA: 0s - loss: 1.0389 - accuracy: 0.6250